# Mixing Station Workspace

***

## Run Hardware Checks

Run the below code block to create an instance of a device, using the hardcoded values and com port addresses provided [here](data/devices/mixing_stations.json). This code will also run through some checks to ensure all necessary hardware is connected and happy.

In [1]:
%load_ext autoreload
%autoreload 2

from src.robot_controller import hardware_scheduler

device = hardware_scheduler.scheduler(device_name="microtron_02", csv_filename="electrolyte_recipe.csv", home=False)

2025-03-12 15:26:29 INFO: Located device data for microtron_02.
2025-03-12 15:26:29 INFO: Configuring fluid handling kit serial port..
2025-03-12 15:26:29 INFO: Attempting to open fluid handling kit serial port..
2025-03-12 15:26:29 INFO: Serial connection to fluid handling kit established.
2025-03-12 15:26:29 INFO: No serial connection to mass balance established.
2025-03-12 15:26:29 INFO: Configuring temperature controller serial port..
2025-03-12 15:26:29 INFO: Attempting to open temperature controller serial port..
2025-03-12 15:26:29 INFO: Serial device located: 18245 TC-XX-PR-59 REV2.6
2025-03-12 15:26:29 INFO: Serial connection to temperature controller established.
2025-03-12 15:26:29 INFO: Temperature regulator PID mode successfully configured.
2025-03-12 15:26:29 INFO: Temperature regulator dead band settings successfully configured.
2025-03-12 15:26:29 INFO: Temperature regulator voltage alarm settings successfully configured.
2025-03-12 15:26:29 INFO: Temperature regulator 

## Run Experiment

Running the below code block will begin a single experiment that creates an electrolye mixture based on the volumes of constitutent electrolytes given in the [electrolyte receipe csv file](data/recipes/electrolyte_recipe.csv).

In [ ]:
device.run(temp=25)
device.clean()

***

## Some Theory

The mixing station pulls information on each constituent electrolyte from a CSV file when creating a new electrolyte mixture. The constituents are numbered, which corresponds to the location of their container in the physical workspace. See the required information for each constituent below..

| # | Name | Dose Volume (uL) | Container Volume (mL) | Density (g/mL) | Aspirate Scalar | Aspirate Speed (uL/s) |
| --- | --- | --- | --- | --- | --- | --- |

Based on the [literature](https://www.theleeco.com/uploads/2023/06/AN049-Pipetting-Disc-Pump-Application-Note-1.pdf), we can expect the aspirate constant to be roughly equal to the system pressure $P_r$ divided by the reservoir volume $V_r$. The system pressure can be assumed equal to atmospheric pressure $\approx$ 1000mbar and the chosen reservoir volume is 2500uL $\implies$ the constant should be about **0.4mbar/uL**.

$$ \Delta P_r = \frac{P_s}{V_r} V_{asp} $$ 

For more viscous liquids, this value may increase based on the ohm's law equivalent of fluid flow through a pipe (the Hagen-Poiseuille equation). To compensate for this, an extra variable is included to slow down the rate of aspiration in an attempt to lower the pressure change required to aspirate more viscous fluids. Along with the response time of the pressure PID controller, the aspirate speed will determine how quickly the pressure of the disc pump changes. A typical value would be **100uL/s**.

$$ \Delta P = \frac{8 \mu L}{\pi R^4} Q $$ 

***

## Tuning Aspiration Variables

**The aspirate speed can be set to zero, to jump straight to the aspiration pressure, for lowest viscosity liquids.**

You can run the below code blocks to perform automatic tuning, where the machine will loop through the parameter ranges and use mass balance data to measure the errors. Tuning therefore requires the mass balance to be connected and positioned in the same space location as the mixing chamber.

The aspirate scalar is a factor to be multiplied by the default *mbar/uL* of water. The aspirate scalar may be greater than 1.0 for more viscous liquids.

In [ ]:
%matplotlib inline
device.tune(pot_number = 8, aspirate_scalars = [0.98, 1.02], aspirate_volume = [100.0, 1000.0], container_volume = 38.0, density = 1.0, N = 3, M = 5)

In [ ]:
%matplotlib inline
device.plot_aspiration_results("data/results/aspiration_tuning_results.csv")

***

### Run life test

Run *N* number of experiments in succession, using the parameters defined in the [life test csv file](data/recipes/life_test.csv).

In [ ]:
device.run_life_test(N=10)

***

## Squidstat Analysis Mode

The analysis mode of the squidstat is set as a [hardcoded value](data/devices/hardcoded_values.json), as an integer correspnding to the modes shown in the following code block. The specifics of each mode can be changed [here](src/robot_controller/admiral.py).

The cell constant used in the calculations that follow the Squidstat measurements is also set via hardcoded value.

In [3]:
list(device.test_cell.squid.modes)

['0. EIS_Potentiostatic \n',
 '1. Cyclic_Voltammetry \n',
 '2. Constant_Current \n',
 '3. Constant_Potential \n',
 '4. Constant_Power \n',
 '5. Constant_Resistance \n',
 '6. DC_Current_Sweep \n',
 '7. DC_Potential_Sweep \n',
 '8. Differential_Pulse_Voltammetry \n',
 '9. Normal_Pulse_Voltammetry \n',
 '10. Square_Wave_Voltammetry \n',
 '11. EIS_Galvanostatic \n',
 '12. Open_Circuit_Potential \n']

## Run Quick Squidstat Analysis

In [ ]:
from robot_controller import test_cell

test_cell = test_cell.measurements(squid_port="COM14", temp_port="", squid_sim=False, temp_sim=True)
test_cell.squid.take_measurements(identifier="test")
results = test_cell.get_impedance_properties(identifier="test", plot=True)

INFO:root:No serial connection to temperature controller established.
INFO:root:Resetting AC and DC dataframes..
INFO:root:Serial connection to Squidstat established.
INFO:root:Ohmic Resistance calculated as 118199.12899999999Ohms (Dataset: test).
INFO:root:Ionic Conductivity calculated as 0.85mS/cm (Dataset: test).
INFO:root:Saving EIS plot (Dataset test)..
